# Soil moisture mapping module

In [1]:
#set user libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Image
from ipywidgets import HBox, Label
import subprocess
import getpass
import os
import sys
import ee
import itertools
from IPython.core.display import HTML 
ee.Initialize()

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



# Introduction

In [2]:
user = getpass.getuser()
print(f"Welcome {user}!, run this application to create soil moisture maps from Sentinel 1 imagery.")

Welcome dguerrero!, run this application to create soil moisture maps from Sentinel 1 imagery.


# Set user variables

## Define the area of interest
### Provide the link to a Google Earth Engine asset and define the column name and value to select

In [3]:
aoi =  widgets.Text(
    value='users/yelenafinegold/107_phu_merged',
    placeholder='Enter your asset ID here',
    disabled=False
)
ui1 = HBox([Label('Google Earth Engine asset ID: '),aoi])
display(ui1)

In [4]:
aoi_ee = ee.FeatureCollection(aoi.value)
col_ee = ee.Feature(aoi_ee.first()).propertyNames().getInfo()

column_name =  widgets.Select(
    value='id',
    options=col_ee,
    placeholder='column name from the GEE asset',
    disabled=False
)
ui2 = HBox([Label('Column name: '),column_name])
display(ui2)

In [5]:
id_values_ee = aoi_ee.distinct(column_name.value).aggregate_array(column_name.value).getInfo()
column_value = widgets.Select(
    options=id_values_ee,
    placeholder='column value from the GEE asset',
    disabled=False
)
ui3 = HBox([Label('Column value: '),column_value])
display(ui3)


In [6]:
# Modify the Year, Month, and Day parameters as desired
year = widgets.SelectMultiple(
    options=range(2014,2020),
    value=[2015,2016,2017,2018,2019],
    description='Year',
    disabled=False
)

display(year)

month = widgets.SelectMultiple(
    options=[('January',1),('February',2),('March',3),('April',4),('May',5),
             ('June',6),('July',7),('August',8),('September',9),('October',10),
             ('November',11),('December',12)
    ],
    value=[4,10],
    description='Month',
    disabled=False
)

display(month)

day = widgets.SelectMultiple(
    options=range(1,32),
    value=[1],
    description='Day',
    disabled=False
)

display(day)


SelectMultiple(description='Year', index=(1, 2, 3, 4, 5), options=(2014, 2015, 2016, 2017, 2018, 2019), value=…

SelectMultiple(description='Month', index=(3, 9), options=(('January', 1), ('February', 2), ('March', 3), ('Ap…

SelectMultiple(description='Day', index=(0,), options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …

# Run the PYSMM code for a specified AOI - will iterate over all combinations of Year / Month / Day sequentially.

In [11]:
study_area = aoi_ee.filterMetadata(column_name.value,'equals', float(column_value.value)).geometry().bounds().coordinates()
coords = study_area.get(0).getInfo()

ll = coords[0]
ur = coords[2]
minlon = ll[0]
minlat = ll[1]
maxlon = ur[0]
maxlat = ur[1]
button = widgets.Button(description='Run SMM')
# button, output, function and linkage
outt = widgets.Output()

year1 = list(year.value)
month1 = list(month.value)
day1 = list(day.value)
attribute_value = column_value.value

print(f'Asset: {aoi.value}')
print(f'Column: {column_name.value}')
print(f'Column value: {attribute_value}')
print(f'Years: {year1}' )
print(f'Months: {month1}')
print(f'Days: {day1}' )

def on_button_clicked(b):
    with outt:
        clear_output()
        print('Processing and downloading soil moisture maps. This takes a long time')
        print(f'Using the feature: {column_value.value} and the aoi: "{aoi.value}"')
#         !mv ./nohup.out ./nohup_backup.out
        py = sys.executable
        !bash ~/sepal_pysmm/scripts/run_pysmm.sh $py "$year1" "$month1" "$day1" $minlon $minlat $maxlon $maxlat $attribute_value
        print('done!')
        
button.on_click(on_button_clicked)
widgets.VBox([button,outt])


Asset: users/yelenafinegold/107_phu_merged
Column: id
Column value: 1120
Years: [2015, 2016, 2017, 2018, 2019]
Months: [4, 10]
Days: [1]
